In [62]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import os
# import tensorflow as tf
import numpy as np
import parser
import math
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn.model_selection import cross_val_score

In [19]:
taskAmount = 128
machineAmount = 4
taskHeterogeneity = 0
machineHeterogeneity = 0
consistencyType = 0
accuracyScores = []
classifiers = []

dimension = taskAmount * machineAmount

# Referencia: https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw
ns = 600 # Cantidad de instancias de entrenamiento
ni = dimension
no = 1 # Cantidad de salidas
alpha = 2
hiddenLayerAmount = int(math.ceil(ns / (alpha * (ni + no))))
# Cada capa oculta tiene una cantidad de neuronas intermedia entre la cantidad de salidas y entradas
# Se pasa a una tupla para mandarlo al constructor del MLPClassifier
hiddenLayerNeuronAmount = tuple([int(math.ceil((taskAmount - no) / 2))] * hiddenLayerAmount)
# Specify that all features have real-value data
# feature_columns = [tf.contrib.layers.real_valued_column("", dimension=dimension)]

In [20]:
print(hiddenLayerAmount)
print(hiddenLayerNeuronAmount)

1
(64,)


In [66]:
# X = [[0., 0.], [1., 1.]]
# y = [0, 1]
# clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
#                     hidden_layer_sizes=hiddenLayerNeuronAmount, random_state=1)

# clf.fit(X, y)   


# clf.predict([[2., 2.], [-1., -2.]])


# for i in range(0, taskAmount):
#     # TODO cambiar n_classes a algo parametrico para machine amount    
#     classifier = MLPClassifier(solver='lbfgs', alpha=1e-5,
#                     hidden_layer_sizes=(), random_state=1)
#     classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
#                                                 hidden_units=[1],
#                                                 n_classes=machineAmount,
#                                                 model_dir='/tmp/classifiers/' + str(taskAmount) + 'x' + str(machineAmount) \
#                                                + '-' + str(taskHeterogeneity) + str(machineHeterogeneity) + str(consistencyType) \
#                                                + '/' + str(i))
#     classifiers.append(classifier)


In [70]:
bla = './ejemplos-entrenamiento-unificados/128x4-000/'
TRAINING_FILE = bla + 'training/' + str(0) + '.csv'
TEST_FILE = bla + 'test/' + str(0) + '.csv'

trainingData = pd.read_csv(TEST_FILE, header=None, delimiter=',')
testData = pd.read_csv(TRAINING_FILE, header=None, delimiter=',')

In [71]:
dfTraining = pd.DataFrame(trainingData)
yTraining = dfTraining.iloc[:,-1]

dfTest = pd.DataFrame(testData)
yTest = dfTest.iloc[:,-1]


# Se entrena clasificador con datos
clf.fit(dfTraining.iloc[:,:-1],yTraining)

# cross_val_score(ann_list[0], training_list[0].iloc[:,:-1], training_list[0].iloc[:,-1], cv=5, scoring='accuracy')
# cross_val_score(clf, df.iloc[:,:-1], df.iloc[:,-1], cv=5, scoring='accuracy')



# #este será un array que contendrá cada dataframe de entrenamiento y su valor objetivo
# #inicializo la lista de dataframes vacíos
# training_list = []
# for i in range(0,12):
#     data_object = pd.DataFrame()
#     training_list.append(data_object)

# for j in range(0,len(df)):
#     tupla_resultado = y[j]
#     tupla_resultado = tupla_resultado.replace("]", "")
#     tupla_resultado = tupla_resultado.replace("[", "")
#     tupla_resultado = tupla_resultado.replace(",", "")
#     array_numbers = tupla_resultado.split(" ")
#     array_numbers = map(int,array_numbers)

#     time_matrix = df.iloc[j,:-1]
#     time_matrix = time_matrix.tolist()

#     for i in range(0,len(array_numbers)):
#         time_copy = copy.deepcopy(time_matrix)
#         time_copy.append(array_numbers[i])
#         training_list[i] = training_list[i].append(pd.Series(time_copy),ignore_index=True)
# # training_list es una lista donde cada lugar en el índice contienen la información de entrenamiento para cada tarea 
# # en particular. Es decir, training_list[0], contiene los datos que serán utilizados para entrenar el algoritmo que 
# # analizará el comportamiento de la tarea0 y devolverá un valor de máquina para esa tarea.


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(64,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [130]:
# df.iloc[:,:-1]
results = []
for i in range(0,len(dfTest)):
    # Clasifico cada instancia del archivo de test, y agrego la clasificacion
    # a un array de resultados
    results.append(clf.predict(dfTest.iloc[i][:-1].values.reshape(1,-1)))
from sklearn.metrics import accuracy_score
print("Precision de clasificador: " + str(accuracy_score(yTest, results)))

Precision de clasificador: 0.513333333333


In [108]:
# for i in range(0, taskAmount):
#     # Build 3 layer DNN with 10, 20, 10 units respectively.
#     # TODO cambiar n_classes a algo parametrico para machine amount    
#     classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns,
#                                                 hidden_units=[1],
#                                                 n_classes=machineAmount,
#                                                 model_dir='/tmp/classifiers/' + str(taskAmount) + 'x' + str(machineAmount) \
#                                                + '-' + str(taskHeterogeneity) + str(machineHeterogeneity) + str(consistencyType) \
#                                                + '/' + str(i))
#     classifiers.append(classifier)

# for i in range(0, taskAmount):
#     # Data sets
#     bla = './ejemplos-entrenamiento-unificados/128x4-000/'
#     TRAINING_FILE = bla + 'training/' + str(i) + '.csv'
#     TEST_FILE = bla + 'test/' + str(i) + '.csv'

#     # Load datasets.
#     training_set = tf.contrib.learn.datasets.base.load_csv_without_header(
#         filename=TRAINING_FILE,
#         target_dtype=np.int,
#         features_dtype=np.float32)
#     test_set = tf.contrib.learn.datasets.base.load_csv_without_header(
#         filename=TEST_FILE,
#         target_dtype=np.int,
#         features_dtype=np.float32)
    
#     # Fit model
#     classifiers[i].fit(x=training_set.data, y=training_set.target, steps=1000)
#     accuracy_score = classifiers[i].evaluate(x=test_set.data, y=test_set.target)["accuracy"]
#     accuracyScores.append(accuracy_score)    
                                                                             
# # for i in range(0, taskAmount):
# #     print('Accuracy for output {}: {0:f}'.format(i, accuracyScores[i]))
# promedio = 0.
# scoreAmount = len(accuracyScores)
# for i in range(0,scoreAmount):
#     promedio += accuracyScores[i]
# promedio /= scoreAmount
# print ('El promedio de accuracy es de {}'.format(promedio))

600

In [112]:
clf.predict(dfTest.iloc[i][:-1].values.reshape(1,-1))[0]

0

In [118]:
list(yTest)


[0,
 2,
 0,
 0,
 0,
 2,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 2,
 0,
 0,
 1,
 0,
 1,
 3,
 2,
 0,
 0,
 2,
 0,
 1,
 0,
 2,
 1,
 0,
 3,
 2,
 0,
 0,
 3,
 1,
 0,
 0,
 3,
 0,
 0,
 1,
 0,
 3,
 0,
 0,
 3,
 0,
 0,
 2,
 2,
 0,
 1,
 0,
 1,
 0,
 3,
 2,
 1,
 0,
 1,
 2,
 2,
 2,
 2,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 2,
 0,
 0,
 0,
 2,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 2,
 1,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 0,
 0,
 3,
 0,
 1,
 0,
 1,
 0,
 3,
 0,
 1,
 3,
 0,
 0,
 0,
 3,
 0,
 0,
 1,
 2,
 1,
 0,
 2,
 3,
 0,
 0,
 1,
 1,
 0,
 0,
 2,
 0,
 2,
 3,
 0,
 1,
 0,
 0,
 2,
 1,
 0,
 0,
 2,
 0,
 1,
 0,
 3,
 3,
 1,
 2,
 0,
 0,
 1,
 2,
 2,
 3,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 2,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 3,
 2,
 1,
 3,
 1,
 2,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 2,
 0,
 0,
 1,
 2,
 0,
 0,
 3,
 2,
 1,
 0,
 3,
 1,
 0,
 0,
 3,
 1,
 0,
 0,
 3,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 3,
 1,
 0,
 2,
 0,
 1,
 2,
 3,
 0,
 1,


In [121]:
results

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
